# Time series classification - Multilayer perceptron

# 1 Load Python modules

In [ ]:
import time

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

import tensorflow as tf # Fast numerical computation for machine learning, computations on GPU or CPU
import tensorflow.keras as keras # High-level interface to TensorFlow, making it easier to create neural networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

# General settings and variables
sns.set_style('whitegrid')
model_palette = ['rebeccapurple', 'mediumspringgreen']

class_names = ['cement', 'carpet']
class_colors = ['darkorange', 'steelblue']

# 2 Functions
Some functions, for convenience.

In [ ]:
def load_data_and_labels(filename):
    ''' Load the dataset file and return the data and labels '''
    # Load the dataset
    url_root = 'https://raw.githubusercontent.com/jarusgnuj/ai-ml-wksh/master/data/UCR_TSC_archive/SonyAIBORobotSurface1_IoC'
    url = url_root+'/'+filename
    robot_df = pd.read_csv(url, sep='\t', header=None)
    print('Loaded from', url)
    robot_data = robot_df.values
    
    # Separate out the data and labels
    labels = robot_data[:,0].astype(int)
    data_samples = robot_data[:,1:]
    
    # Print information
    print('The shape of the data matrix is', data_samples.shape)
    print('Number of samples of class 0', (labels == 0).sum())
    print('Number of samples of class 1', (labels == 1).sum())

    return data_samples, labels
    
    
def plot_comparison(data_train, labels_train, data_test, labels_test, test_sample):
    ''' Plot the given test sample alongside a few training samples of the same class '''
    # Determine the true class of the given sample
    print('Test sample', test_sample, 'true class', str(labels_test[test_sample]), class_names[labels_test[test_sample]])  
    true_class = labels_test[test_sample]

    # Plot data samples that are in the same class
    fig, ax = plt.subplots()
    count = 0
    for i in range(100):
        if labels_train[i] == true_class:
            plt.plot(data_train[i], color=class_colors[labels_train[i]])
            print('Training sample', i, 'class', str(labels_train[i]), class_names[labels_train[i]])
            count = count + 1
            if count > 4:
                break
    plt.ylim([-3.5, 3.5])
    plt.title('Walking on  '+class_names[true_class])
    ax.set_ylabel('Accelerometer data')
    ax.set_xlabel('Data point number')

    # Plot the test data sample
    plt.plot(data_test[test_sample], color='darkred')
    
    
def plot_loss(log):
    ''' Plot the loss recorded in the log during model training '''
    ax = log[['loss', 'val_loss']].plot(title='Loss function during training', color=model_palette)
    ax.set_xlabel("Model training epoch")
    ax.set_ylabel("Loss")
    ax.legend(["training", "validation"]);
    
    
def plot_accuracy(log):
    ''' Plot the accuracy recorded in the log during model training '''
    ax = log[['acc', 'val_acc']].plot(title='Accuracy during training', color=model_palette)
    ax.set_xlabel("Model training epoch")
    ax.set_ylabel("Accuracy")
    ax.legend(["training", "validation"]);

# 3 Load the development dataset

In [ ]:
filename = 'SonyAIBORobotSurface1_IoC_DEV.txt'
data, labels = load_data_and_labels(filename)

# 4 Split the development dataset into training and test datasets
We have already reserved 100 samples for our final test set. This leaves us with 444 data samples in this development phase.

How much data do you want to use for training the model and how much for testing in this development phase? 


![The development dataset is split into two, unequal sets](images/train_test_split.png "Title")

Considerations -
+ If you test on only 10 samples, would you be confident of the result?
+ How long does it take to train the model if the training set contains 20 samples, as compared to 400 samples?
+ Does model accuracy continue to improve as training set size is increased?

Rough guide -
+ If the dataset contains fewer than 1 million samples, then a typical training:test:final test split would be 80%:10%:10% or 60%:20%:20% 
+ If the dataset is larger, over a million samples, then a practitioner might choose to use around 10,000 samples for test, 10,000 samples for final test and the remainder for training. Or they might split the data 99.5%:0.25%:0.25%

Note -
+ "Test dataset" and the "validation dataset" refer to the same dataset. We train the model using the training dataset and use the test dataset to measure validation accuracy.

In [ ]:
test_size = 100 ### CHANGE PARAMETER HERE ###

data_train, data_test, labels_train, labels_test = train_test_split(
    data, labels, test_size=test_size, random_state=21, stratify=labels)

print('The shape of train_data is', data_train.shape)
print('The shape of test_data is', data_test.shape)
print('Training data:')
print('Number of samples of class 0', (labels_train == 0).sum())
print('Number of samples of class 1', (labels_train == 1).sum())
print('Test data:')
print('Number of samples of class 0', (labels_test == 0).sum())
print('Number of samples of class 1', (labels_test == 1).sum())

# 5 Multilayer Perceptron
Create a multilayer perceptron (MLP) model.

## 5.1 MLP neurons and connections
![Visualising the MLP model. 70 inputs. 2 hidden layers - 16 neurons in the first layer, 8 in the second. One neuron in the output layer](images/mlp.png "MLP architecture")

In [ ]:
# The size of the input vector
input_dim = data_train.shape[1]


def build_model(print_summary=False):
    ''' Return a model with randomly initialised weights '''
    model = Sequential([
        Dense(8, input_dim=input_dim, activation='relu', name='Layer1'), 
        Dense(4, activation='relu', name='Layer2'), 
        Dense(1, activation='sigmoid', name='OutputLayer')
    ])

    optimizer = keras.optimizers.Adam() 
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    if print_summary:
        print(model.summary())
    return model


model = build_model(True)

## 5.2 Test the untrained MLP
Let's test the model before we train it.

In [ ]:
result = model.evaluate(data_test, labels_test, batch_size=5)
print('Pre-training, validation accuracy is', result[1])

# 6 Train the MLP
batch_size - how many data samples to input to the MLP in one go.


epochs - how many times the training process will iterate through the entire training set.

Detail
+ For the first step of training, a batch of samples from the training dataset are input the to MLP. 
  + The number of samples in the batch is set by the batch_size parameter.
+ The loss function's value is calculated and the MLP's weights are updated in a way that is expected to reduce the loss value in the next step. 
+ In the second step, the next batch of samples is input to the MLP. The loss value is calculated and the weights are updated.
+ These batch calculations and updates continue until all of the samples in the training dataset have been used. 
  + This completes one epoch.
+ The subsequent epochs similarly cycle through the training dataset batch by batch.
+ Training stops when the number of epochs, a parameter set by the user below, has been reached.

Batch size is typically set to a power of two (2, 4, 8, 16, ...) to best utilse the memory size of a CPU or GPU.

In [ ]:
batch_size = 8 ### CHANGE PARAMETER HERE ### 
epochs = 20 ### CHANGE PARAMETER HERE ###

model = build_model() # This re-initialises the model with random weights each time before we train it.

# Train
start = time.time()
hist = model.fit(data_train, labels_train, batch_size=batch_size, epochs=epochs, 
                 validation_data=(data_test, labels_test), verbose=1)
end = time.time()
log = pd.DataFrame(hist.history) 
print('Training complete in', round(end-start), 'seconds')

In [ ]:
# Use the trained model to classify the test dataset.
result = model.evaluate(data_test, labels_test, batch_size=batch_size)
print('Validation accuracy:\t', result[1])
print('Validation loss:\t', result[0])
print('test_size:\t', test_size)
print('batch_size:\t', batch_size)
print('epochs:\t\t', epochs)

# 7 Exercise 2a : Variability
Rerun the model training and the model evaluation in the two cells above. Each time, note down (in the cell below) the validation accuracy. How much does it vary?

### User comments
Enter your result for each run below -
+ Run 1: validation accuracy = 
+ Run 2: validation accuracy =
+ Run 3: validation accuracy =
+ ...

# 8 Model training progress
We can plot model performance as training progresses using the training log.

## 8.1 Plot the loss

In [ ]:
# Plot the training log's loss data.
plot_loss(log)

## 8.2 Plot the accuracy

In [ ]:
# Plot the training log's accuracy data.
plot_accuracy(log)

## 8.3 Exercise 2b : Training choices
The cells above include some numbers that you can change -
+ test_size 
+ epochs
+ batch_size

They are indicated with ### CHANGE PARAMETER HERE ###. Go ahead and change these and rerun the cells in order to answer the following questions.
+ How much of the development dataset do you think you need use for training, how much for testing?
+ How many training epochs are needed?
+ What batch size works well?

Note your choices below.

### User comments
Enter your own notes below.
Good choices of parameters are:-
+ test_size 
+ epochs
+ batch_size

# 9 Make predictions using the trained MLP
This trained model can now be used to classify data samples. These could be samples from our test set, our final test set or completely new samples.

In machine learning, the class that is output by the model is called a "prediction". A data sample is input to the model and the model returns its "predicted class". If the model is correct, this predicted class will match data sample's "true" class.


In the cell below we will select a single test sample and ask the model to 'predict' its class. The model gives you the probability that this sample belongs to class 1 (carpet). 

Typically, if the probability is above 0.5, we classify the sample as belonging to class 1. If the probability is 0.5 or lower, we classify the sample as belonging to class 0.

+ What class does the model assign to this sample?
+ What is its true class?

In [ ]:
sample_num = 5 ### CHANGE PARAMETER HERE ###
data_sample = data_test[sample_num]
data_sample = np.array( [data_sample,] ) # Convert the data sample into the shape expected by the MLP
probability = model.predict(data_sample)
print('Model: probability of belonging to class 1:', probability[0][0])
print('Predicted class:\t', (np.round(probability)[0][0].astype(int))) # \t inserts a tab space into the text
print('True class:\t\t', labels_test[sample_num])

## 9.1 Optional : Batch predictions
You can ask the model to make predictions on set of data samples.

In [ ]:
labels_probability = model.predict_on_batch(data_test)
labels_predicted_class = np.round(labels_probability).flatten()
print('Some of the test results:')
print('True', labels_test[:23])
print('Pred', labels_predicted_class[:23].astype(int))

# 10 Optional : Exercise 2c : Insight into model performance
+ Identify a test data sample that the model classifies incorrectly.
+ Compare this sample to training data in the same class.
+ How do they compare?
  + What might this tell you about either the data, the training or about the model's ability?
  
  
Repeat your analysis with a few more samples.

Continue on to the next optional section if you wish.

In [ ]:
test_sample = 6 ### CHANGE PARAMETER HERE ###
plot_comparison(data_train, labels_train, data_test, labels_test, test_sample)

# 11 Optional : Model prediction probabilities
Find some test data samples where the model made an incorrect classification and predicted a high probability that the sample belongs to that class. E.g. a sample of true class 0 where the model predicted a probability of 0.95 that the sample belongs to class 1.


Continue exercise 2c focusing on these samples.

In [ ]:
x = np.arange(labels_probability.shape[0])
class_cmap = matplotlib.colors.ListedColormap(class_colors)
fig, ax = plt.subplots()
plt.scatter(np.arange(labels_probability.shape[0]), labels_probability, linestyle='None', marker='x', 
            c=labels_test, cmap=class_cmap)
plt.title('Orange: true class 0 (cement)\nBlue: true class 1 (carpet)')
ax.set_xlabel('Test sample number')
ax.set_ylabel('Model: probability of belonging to class 1');